In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**1. Kütüphaneler**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

**2. Veri Setini Yükleme**

In [ ]:
!kaggle datasets download -d alxmamaev/flowers-recognition

# İndirilen veri setini açma
import zipfile

with zipfile.ZipFile('flowers-recognition.zip', 'r') as zip_ref:
    zip_ref.extractall('flowers-recognition')

# İndirilen veri setinin dosya yapısını kontrol etme
os.listdir('flowers-recognition')


**3. Veri Ön İşleme**

In [ ]:
#Veri Setinin Hazırlanması

# Verileri yüklemek için dizinleri ayarlanması
base_dir = 'flowers-recognition/flowers'
categories = os.listdir(base_dir)

# Veri artırma
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



# Görselleri ve etiketleri saklamak için listeler oluşturma
images = []
labels = []

for category in categories:
    category_path = os.path.join(base_dir, category)
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(128, 128))
        img = tf.keras.preprocessing.image.img_to_array(img)
        images.append(img)
        labels.append(category)

# Verileri numpy dizilerine dönüştürme
images = np.array(images)
labels = np.array(labels)

# Etiketleri kategorik olarak kodlama
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = tf.keras.utils.to_categorical(labels, num_classes=len(categories))

# Verileri eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Verilerin boyutlarını yazdırma
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

**4.  Görselleştirme**

In [ ]:
# Veri seti içindeki görüntülerin boyutlarını yazdırma
print("Görüntülerin boyutu:", X_train[0].shape)

In [ ]:
# Görüntüleri görselleştirme
plt.figure(figsize=(8, 8))
for i in range(10):
    plt.subplot(3, 5, i + 1)
    plt.imshow(X_train[i].astype('uint8'))
    plt.title(categories[np.argmax(y_train[i])])
    plt.axis('off')
plt.show()

**5. Modeli Eğitme** 

**5. Model Oluşturma**

In [ ]:
# CNN Modelini oluşturma
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')
])

# Modeli derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Eğitim sürecinin doğruluk ve kayıp grafikleri
plt.figure(figsize=(12, 6))

In [ ]:
# Eğitim ve doğrulama doğruluğu
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Eğitim ve Doğrulama Doğruluğu')
plt.xlabel('Epok')
plt.ylabel('Doğruluk')
plt.legend()

In [ ]:
# Eğitim ve doğrulama kaybı
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Eğitim ve Doğrulama Kaybı')
plt.xlabel('Epok')
plt.ylabel('Kayıp')
plt.legend()

plt.show()

In [ ]:
# Görüntüleri düzleştirme
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Verileri normalize edin
scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

# Etiketleri 1D diziye dönüştürün
y_train_flat = np.argmax(y_train, axis=1)
y_test_flat = np.argmax(y_test, axis=1)

# Model sonuçlarını saklamak için bir sözlük oluşturun
model_results = {}

In [ ]:
# SVM modeli
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_flat, y_train_flat)
y_pred_svm = svm_model.predict(X_test_flat)

# Sonuçları değerlendirme
svm_accuracy = accuracy_score(y_test_flat, y_pred_svm)
svm_precision = precision_score(y_test_flat, y_pred_svm, average='weighted')
svm_recall = recall_score(y_test_flat, y_pred_svm, average='weighted')
svm_f1 = f1_score(y_test_flat, y_pred_svm, average='weighted')

model_results['SVM'] = {
    'accuracy': svm_accuracy,
    'precision': svm_precision,
    'recall': svm_recall,
    'f1_score': svm_f1
}

In [ ]:
# Logistic Regression modeli
lr_model = LogisticRegression(max_iter=500)
lr_model.fit(X_train_flat, y_train_flat)
y_pred_lr = lr_model.predict(X_test_flat)

# Sonuçları değerlendirme
lr_accuracy = accuracy_score(y_test_flat, y_pred_lr)
lr_precision = precision_score(y_test_flat, y_pred_lr, average='weighted')
lr_recall = recall_score(y_test_flat, y_pred_lr, average='weighted')
lr_f1 = f1_score(y_test_flat, y_pred_lr, average='weighted')

model_results['Logistic Regression'] = {
    'accuracy': lr_accuracy,
    'precision': lr_precision,
    'recall': lr_recall,
    'f1_score': lr_f1
}

In [ ]:
# Random Forest modeli
rf_model = RandomForestClassifier()
rf_model.fit(X_train_flat, y_train_flat)
y_pred_rf = rf_model.predict(X_test_flat)
# Sonuçları değerlendirme
rf_accuracy = accuracy_score(y_test_flat, y_pred_rf)
rf_precision = precision_score(y_test_flat, y_pred_rf, average='weighted')
rf_recall = recall_score(y_test_flat, y_pred_rf, average='weighted')
rf_f1 = f1_score(y_test_flat, y_pred_rf, average='weighted')

model_results['Random Forest'] = {
    'accuracy': rf_accuracy,
    'precision': rf_precision,
    'recall': rf_recall,
    'f1_score': rf_f1
}

In [ ]:
# Model sonuçlarını yazdırma
for model_name, metrics in model_results.items():
    print(f"Model: {model_name}")
    for metric_name, value in metrics.items():
        print(f"{metric_name.capitalize()}: {value:.4f}")
    print("\n")